In [ ]:
import pickle
import TextEmbedding 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import make_scorer
from keras.layers import Dense,Embedding,Dropout,Flatten
from keras.optimizers import Adam

from tensorflow.keras.models import Sequential
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.utils import to_categorical

In [ ]:
X_train = pickle.load(open('FinalData\X_train.pkl', 'rb'))
X_test = pickle.load(open('FinalData\X_test.pkl', 'rb'))
y_train = pickle.load(open('FinalData\y_train.pkl', 'rb'))
y_test = pickle.load(open('FinalData\y_test.pkl', 'rb'))

X_train_w2v = pickle.load(open('FinalData\X_train_w2v.pkl', 'rb'))
X_test_w2v = pickle.load(open('FinalData\X_test_w2v.pkl', 'rb'))
y_train_w2v = pickle.load(open('FinalData\y_train_w2v.pkl', 'rb'))
y_test_w2v = pickle.load(open('FinalData\y_test_w2v.pkl', 'rb'))
tfidf_embedding_train, tfidf_embedding_test = TextEmbedding.tfidf(X_train,X_test)
BOW_embedding_train, BOW_embedding_test = TextEmbedding.BagOfWord(X_train,X_test)
w2v_embedding_train, w2v_embedding_test = TextEmbedding.Word2Vector(X_train,X_test)


In [ ]:
list_topic ={
    "thoi-su": 0,
    "kinh-doanh": 1,
    "khoa-hoc": 2,
    "giai-tri": 3,
    "the-thao": 4,
    "phap-luat": 5, 
    "giao-duc": 6,
    "suc-khoe": 7,
    "doi-song": 8,
    "du-lich":  9
}
for i in range (len(y_train)):
    for x, y in list_topic.items():
        if y_train[i] == x:
            y_train[i] = y 
for i in range (len(y_test)):
    for x, y in list_topic.items():
        if y_test[i] == x:
            y_test[i] = y

In [ ]:
def random_forest(X_train, y_train, X_test, y_test, maxdepth, number_estimators):
    rdf = RandomForestClassifier(n_estimators= number_estimators, max_depth = maxdepth, random_state=0)
    rdf.fit(X_train, y_train)
    test_predictions = rdf.predict(X_test)
    f1 = f1_score(y_test, test_predictions, average='macro')
    recall = recall_score(y_test, test_predictions, average='macro')
    accuracy = accuracy_score(y_test, test_predictions)
    return accuracy, recall, f1

GET BEST SET OF PARAMETERS (USING BOW)

In [ ]:
values = [50, 60, 70, 80, 90, 100, 110]  
accuracy_values = []
recall_values = []
f1_values = []

for value in values:
    accuracy, recall, f1 = random_forest(BOW_embedding_train, y_train, BOW_embedding_test, y_test, value, 500)
    accuracy_values.append(accuracy)
    recall_values.append(recall)
    f1_values.append(f1)


bar_width = 0.2
print(accuracy_values)
print(recall_values)
print(f1_values)

bar_positions = np.arange(len(values))

plt.bar(bar_positions, accuracy_values, width=bar_width, color='red', label='Accuracy')
plt.bar(bar_positions + bar_width, recall_values, width=bar_width, color='blue', label='Recall')
plt.bar(bar_positions + 2*bar_width, f1_values, width=bar_width, color='green', label='F1 Score')

plt.xlabel('max_depth')
plt.ylabel('Accuracy')
plt.title('Random forest performance with n_estimators = 500')
plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.22), ncol=3)

plt.xticks(bar_positions + bar_width, values)
plt.yticks(np.arange(0, 1.1, 0.1))

plt.show()



In [ ]:
def plot_grid_search(param, list_params, result):
    values = [params[param] for params in list_params]

    accuracy_values = result

    bar_width = 0.2
    

    bar_positions = np.arange(len(values))

    plt.bar(bar_positions, accuracy_values, width=bar_width, color='red', label='Accuracy')

    plt.xlabel('n_estimators')
    plt.ylabel('Accuracy')
    plt.title('Random Forest Performance Accuracy with max_depth = 80')
    plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.22), ncol=3)

    plt.xticks(bar_positions, values)
    plt.yticks(np.arange(0, 1.1, 0.1))

    plt.show()

In [ ]:
def GridSearch_RandomForest():
    param_grid = {
    # 'max_depth': [50, 60, 70, 80, 90, 100, 110],
    'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
    }
    rf = RandomForestClassifier()
    gs = GridSearchCV(estimator = rf, param_grid = param_grid, 
                            cv = 5, n_jobs = -1, verbose = 1)
    grid_search = gs.fit(BOW_embedding_train, y_train)
    print("Best parameters: ", grid_search.best_params_)
    print("Best score: ", grid_search.best_score_)
    plot_grid_search('n_estimators', grid_search.cv_results_['params'], grid_search.cv_results_['mean_test_score'])

GridSearch_RandomForest()

GET BEST SET OF PARAMETERS (USING TFIDF)

In [ ]:
values = [50, 60, 70, 80, 90, 100, 110]  
accuracy_values = []
recall_values = []
f1_values = []

for value in values:
    accuracy, recall, f1 = random_forest(tfidf_embedding_train, y_train, tfidf_embedding_test, y_test, value, 500)
    accuracy_values.append(accuracy)
    recall_values.append(recall)
    f1_values.append(f1)


bar_width = 0.2
print(accuracy_values)
print(recall_values)
print(f1_values)

bar_positions = np.arange(len(values))

plt.bar(bar_positions, accuracy_values, width=bar_width, color='red', label='Accuracy')
plt.bar(bar_positions + bar_width, recall_values, width=bar_width, color='blue', label='Recall')
plt.bar(bar_positions + 2*bar_width, f1_values, width=bar_width, color='green', label='F1 Score')

plt.xlabel('max_depth')
plt.ylabel('Accuracy')
plt.title('Random forest performance with n_estimators = 500')
plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.22), ncol=3)

plt.xticks(bar_positions + bar_width, values)
plt.yticks(np.arange(0, 1.1, 0.1))

plt.show()



In [ ]:
def plot_grid_search(param, list_params, result):
    values = [params[param] for params in list_params]

    accuracy_values = result

    bar_width = 0.2
    

    bar_positions = np.arange(len(values))

    plt.bar(bar_positions, accuracy_values, width=bar_width, color='red', label='Accuracy')

    plt.xlabel('n_estimators')
    plt.ylabel('Accuracy')
    plt.title('Random Forest Performance Accuracy with max_depth = 80')
    plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.22), ncol=3)

    plt.xticks(bar_positions, values)
    plt.yticks(np.arange(0, 1.1, 0.1))

    plt.show()

In [ ]:
def GridSearch_RandomForest():
    param_grid = {
    # 'max_depth': [50, 60, 70, 80, 90, 100, 110],
    'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
    }
    rf = RandomForestClassifier()
    gs = GridSearchCV(estimator = rf, param_grid = param_grid, 
                            cv = 5, n_jobs = -1, verbose = 1)
    grid_search = gs.fit(tfidf_embedding_train, y_train)
    print("Best parameters: ", grid_search.best_params_)
    print("Best score: ", grid_search.best_score_)
    plot_grid_search('n_estimators', grid_search.cv_results_['params'], grid_search.cv_results_['mean_test_score'])

GridSearch_RandomForest()

GET BEST SET OF PARAMETERS (USING WORD2VEC)

In [ ]:
values = [50, 60, 70, 80, 90, 100, 110]  
accuracy_values = []
recall_values = []
f1_values = []

for value in values:
    accuracy, recall, f1 = random_forest(w2v_embedding_train, y_train, w2v_embedding_test, y_test, value, 500)
    accuracy_values.append(accuracy)
    recall_values.append(recall)
    f1_values.append(f1)


bar_width = 0.2
print(accuracy_values)
print(recall_values)
print(f1_values)

bar_positions = np.arange(len(values))

plt.bar(bar_positions, accuracy_values, width=bar_width, color='red', label='Accuracy')
plt.bar(bar_positions + bar_width, recall_values, width=bar_width, color='blue', label='Recall')
plt.bar(bar_positions + 2*bar_width, f1_values, width=bar_width, color='green', label='F1 Score')

plt.xlabel('max_depth')
plt.ylabel('Accuracy')
plt.title('Random forest performance with n_estimators = 500')
plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.22), ncol=3)

plt.xticks(bar_positions + bar_width, values)
plt.yticks(np.arange(0, 1.1, 0.1))

plt.show()



In [ ]:
def plot_grid_search(param, list_params, result):
    values = [params[param] for params in list_params]

    accuracy_values = result

    bar_width = 0.2
    

    bar_positions = np.arange(len(values))

    plt.bar(bar_positions, accuracy_values, width=bar_width, color='red', label='Accuracy')

    plt.xlabel('n_estimators')
    plt.ylabel('Accuracy')
    plt.title('Random Forest Performance Accuracy with max_depth = 80')
    plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.22), ncol=3)

    plt.xticks(bar_positions, values)
    plt.yticks(np.arange(0, 1.1, 0.1))

    plt.show()

In [ ]:
def GridSearch_RandomForest():
    param_grid = {
    # 'max_depth': [50, 60, 70, 80, 90, 100, 110],
    'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
    }
    rf = RandomForestClassifier()
    gs = GridSearchCV(estimator = rf, param_grid = param_grid, 
                            cv = 5, n_jobs = -1, verbose = 1)
    grid_search = gs.fit(w2v_embedding_train, y_train)
    print("Best parameters: ", grid_search.best_params_)
    print("Best score: ", grid_search.best_score_)
    plot_grid_search('n_estimators', grid_search.cv_results_['params'], grid_search.cv_results_['mean_test_score'])

GridSearch_RandomForest()

COMPARE BOW, TFIDF, WORD2VEC WITH BEST OF SET PARAMETERS 